In [9]:


# ..............................................PART 1 FACE RECOGNITION...............

import face_recognition
import cv2
import os
import glob
import numpy as np

class SimpleFacerec:
    def __init__(self):
        self.known_face_encodings = []
        self.known_face_names = []

        # Resize frame for a faster speed
        self.frame_resizing = 0.25

    def load_encoding_images(self, images_path):
        """
        Load encoding images from path
        :param images_path:
        :return:
        """
        # Load Images
        images_path = glob.glob(os.path.join(images_path, "*.*"))

        print("{} encoding images found.".format(len(images_path)))

        # Store image encoding and names
        for img_path in images_path:
            img = cv2.imread(img_path)
            rgb_img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

            # Get the filename only from the initial file path.
            basename = os.path.basename(img_path)
            (filename, ext) = os.path.splitext(basename)
            # Get encoding
            img_encoding = face_recognition.face_encodings(rgb_img)[0]

            # Store file name and file encoding
            self.known_face_encodings.append(img_encoding)
            self.known_face_names.append(filename)
        print("Encoding images loaded")

    def detect_known_faces(self, frame):
        small_frame = cv2.resize(frame, (0, 0), fx=self.frame_resizing, fy=self.frame_resizing)
        # Find all the faces and face encodings in the current frame of video
        # Convert the image from BGR color (which OpenCV uses) to RGB color (which face_recognition uses)
        rgb_small_frame = cv2.cvtColor(small_frame, cv2.COLOR_BGR2RGB)
        face_locations = face_recognition.face_locations(rgb_small_frame)
        face_encodings = face_recognition.face_encodings(rgb_small_frame, face_locations)

        face_names = []
        for face_encoding in face_encodings:
            # See if the face is a match for the known face(s)
            matches = face_recognition.compare_faces(self.known_face_encodings, face_encoding)
            name = "Unknown"

            # # If a match was found in known_face_encodings, just use the first one.
            # if True in matches:
            #     first_match_index = matches.index(True)
            #     name = known_face_names[first_match_index]

            # Or instead, use the known face with the smallest distance to the new face
            face_distances = face_recognition.face_distance(self.known_face_encodings, face_encoding)
            best_match_index = np.argmin(face_distances)
            if matches[best_match_index]:
                name = self.known_face_names[best_match_index]
            face_names.append(name)

        # Convert to numpy array to adjust coordinates with frame resizing quickly
        face_locations = np.array(face_locations)
        face_locations = face_locations / self.frame_resizing
        return face_locations.astype(int), face_names




In [10]:

# ........................................PART 2 OPEN WEBCAM .......................
    
import cv2
import time
# from simple_facerec import SimpleFacerec

# Encode faces from a folder
sfr = SimpleFacerec()
sfr.load_encoding_images("images/")

# Load Camera
cap = cv2.VideoCapture(0)


while True:
    ret, frame = cap.read()

    # Detect Faces
    face_locations, face_names = sfr.detect_known_faces(frame)
    for face_loc, name in zip(face_locations, face_names):
        y1, x2, y2, x1 = face_loc[0], face_loc[1], face_loc[2], face_loc[3]


        # Get current time 
        current_time = time.strftime("%Y-%m-%d %H:%M:%S", time.localtime())
        # print("Detected Face", face_loc, name)
        print(f"Detected Face {face_loc} {name} at {current_time}")

        cv2.putText(frame, name,(x1, y1 - 10), cv2.FONT_HERSHEY_DUPLEX, 1, (0, 0, 200), 2) # get Face name
        # Display detection time below the face name
        cv2.putText(frame, f"{current_time}", (x1, y2 + 50), cv2.FONT_HERSHEY_DUPLEX, 0.5, (0, 0, 200), 1)

        # cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 0, 200), 4) # 4 pixels
        # cv2.putText(frame, f"{name} - {current_time}", (x1, y1 - 10), cv2.FONT_HERSHEY_DUPLEX, 1, (0, 0, 200), 2)

        cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 0, 200), 4)





    cv2.imshow("Frame", frame)

    # key = cv2.waitKey(1)
    # if key == 27:
    #     break
    key = cv2.waitKey(2)
    if key == ord('q') or key == 27:  # 'q' key or Esc key
        break

cap.release()
cv2.destroyAllWindows()


6 encoding images found.
Encoding images loaded
Detected Face [220 448 428 240] NTAMBARA Etienne at 2024-01-16 01:28:38
Detected Face [220 448 428 240] NTAMBARA Etienne at 2024-01-16 01:28:39
Detected Face [184 484 432 236] NTAMBARA Etienne at 2024-01-16 01:28:49
Detected Face [220 448 428 240] NTAMBARA Etienne at 2024-01-16 01:28:54
Detected Face [240 432 412 256] NTAMBARA Etienne at 2024-01-16 01:28:55
Detected Face [220 448 428 240] NTAMBARA Etienne at 2024-01-16 01:28:56
Detected Face [240 432 412 256] NTAMBARA Etienne at 2024-01-16 01:28:57
Detected Face [220 448 428 240] NTAMBARA Etienne at 2024-01-16 01:28:57
Detected Face [220 448 428 240] NTAMBARA Etienne at 2024-01-16 01:28:58
Detected Face [220 432 392 256] NTAMBARA Etienne at 2024-01-16 01:29:02
Detected Face [220 448 428 240] NTAMBARA Etienne at 2024-01-16 01:29:04
Detected Face [220 448 428 240] NTAMBARA Etienne at 2024-01-16 01:29:04
Detected Face [220 448 428 240] NTAMBARA Etienne at 2024-01-16 01:29:05
Detected Face [2

source codes 
https://pysource.com/2021/08/16/face-recognition-in-real-time-with-opencv-and-python/


https://youtu.be/5yPeKQzCPdI?si=ftccxdgeLwqa8e8N